# Creating new resources

Vortex initialisation

In [1]:
%load_ext ivortex
%vortex
toolbox.active_verbose = False
toolbox.active_now = False

Vortex 1.6.2 loaded ( Thursday 29. August 2019, at 12:07:40 )


## Creating a resource compatible with the *Vortex*  *provider*

For our example, the requirements for the new resource are:

* To be described by *kind*, *model*, *date*, *cutoff*, *number* and *scope* attributes;
* That the *scope* attribute can only take values such as ``surface`` or ``altitude``.

Several versions of this new resource will be created for the purpose of the training session (to describe the different ways of dealing with new resources): in the real world, only one class should be created.

### “Standard” programming model 

In [2]:
from vortex.data.flow import FlowResource
from vortex.syntax.stdattrs import number_deco

class DemoResource1(FlowResource):

    _footprint = [
        number_deco,
        dict(info = 'Description 1',
             attr = dict(
                kind = dict(values = ['demo1']),
                scope = dict(info = 'Scope description',
                             values = ['surface', 'altitude'])))
    ]
    
    @property
    def realkind(self):
        return 'demo'
    
    def namebuilding_info(self):
        """On ajoute l'attribut *scope*."""
        bdict = super(DemoResource1, self).namebuilding_info()
        bdict['src'].append(self.scope)
        return bdict

Here, the **namebuilding_info** method is redefined manually. We can notice the benefits of inheritance (**FlowResource** provided attributes *model*, *dates* and *cutoff*) and of predefined footprints attributes (``number_deco`` provides the *number* attribute).

Let's test the result:

In [3]:
rtest = toolbox.input(date='2019010100', cutoff='assim', model='arpege',
                      kind='demo1', number=5, scope='altitude', nativefmt='grib',
                      # Provider + Container :
                      experiment='ABCD', block='anyblock', namespace='vortex.cache.fr',
                      vapp='arpege', vconf='4dvarfr', incore=True)
rtest[0].location()

'vortex://vortex.cache.fr/arpege/4dvarfr/ABCD/20190101T0000A/anyblock/demo.arpege-altitude.005.grib'

### Use of a class decorator

In [4]:
from vortex.data.flow import FlowResource
from vortex.syntax.stdattrs import number_deco
from vortex.syntax.stddeco import namebuilding_append

@namebuilding_append('src', lambda self: self.scope)
class DemoResource2(FlowResource):

    _footprint = [
        number_deco,
        dict(info = 'Description 2',
             attr = dict(
                kind = dict(values = ['demo2']),
                scope = dict(info = 'Scope description',
                             values = ['surface', 'altitude'])))
    ]
    
    @property
    def realkind(self):
        return 'demo'

Here, the ``namebuilding_append`` class decorator adds the scope attribute to the dictionary returned by **namebuilding_info**...

Let's test the result:

In [5]:
rtest = toolbox.input(date='2019010100', cutoff='assim', model='arpege',
                      kind='demo2', number=5, scope='altitude', nativefmt='grib',
                      # Provider + Container :
                      experiment='ABCD', block='anyblock', namespace='vortex.cache.fr',
                      vapp='arpege', vconf='4dvarfr', incore=True)
rtest[0].location()

'vortex://vortex.cache.fr/arpege/4dvarfr/ABCD/20190101T0000A/anyblock/demo.arpege-altitude.005.grib'

### Creating a predefined attribute

In [6]:
from vortex.data.flow import FlowResource
from vortex.syntax.stdattrs import number_deco
from vortex.syntax.stddeco import namebuilding_append

demoscope = fp.Footprint(info = 'Abstract Scope',
                         attr = dict(scope = dict(info = 'Scope description',
                                                 values = ['surface', 'altitude'])))

demoscope_deco = fp.DecorativeFootprint(
    demoscope,
    decorator = [namebuilding_append('src', lambda self: self.scope), ])

class DemoResource3(FlowResource):

    _footprint = [
        number_deco,
        demoscope_deco,
        dict(info = 'Description 3',
             attr = dict(kind = dict(values = ['demo3'])))
    ]
    
    @property
    def realkind(self):
        return 'demo'

We define a predefined attribute for *scope* (``demoscope`` object) and we associate it with a class decorator (``demoscope_deco`` object). If the *scope* attribute is likely to be used in other resources, this strategy can be cost effective.

Let's test the result:

In [7]:
rtest = toolbox.input(date='2019010100', cutoff='assim', model='arpege',
                      kind='demo3', number=5, scope='altitude', nativefmt='grib',
                      # Provider + Container :
                      experiment='ABCD', block='anyblock', namespace='vortex.cache.fr',
                      vapp='arpege', vconf='4dvarfr', incore=True)
rtest[0].location()

'vortex://vortex.cache.fr/arpege/4dvarfr/ABCD/20190101T0000A/anyblock/demo.arpege-altitude.005.grib'

# Extension of the previous example for the Olive *provider*

In [8]:
from vortex.data.flow import FlowResource
from vortex.syntax.stdattrs import number_deco
# demoscope_deco defined above is reused here

class DemoResource4(FlowResource):

    _footprint = [
        number_deco,
        demoscope_deco,
        dict(info = 'Description 4',
             attr = dict(kind = dict(values = ['demo4'])))
    ]
    
    @property
    def realkind(self):
        return 'demo'
    
    def olive_basename(self):
        return '{0.realkind:s}{0.scope:s}{0.number:04d}'.format(self).upper()

Let's test the result:

In [10]:
rtest = toolbox.input(date='2019010100', cutoff='assim', model='arpege',
                      kind='demo4', number=5, scope='altitude', nativefmt='grib',
                      # Provider + Container :
                      experiment='ABCD', block='anyblock', namespace='olive.cache.fr',
                      vapp='arpege', vconf='4dvarfr', incore=True)
rtest[0].location()

'olive://olive.cache.fr/ABCD/20190101H00A/anyblock/DEMOALTITUDE0005'

It still works with the Vortex *provider*...

In [9]:
rtest = toolbox.input(date='2019010100', cutoff='assim', model='arpege',
                      kind='demo4', number=5, scope='altitude', nativefmt='grib',
                      # Provider + Container :
                      experiment='ABCD', block='anyblock', namespace='vortex.cache.fr',
                      vapp='arpege', vconf='4dvarfr', incore=True)
rtest[0].location()

'vortex://vortex.cache.fr/arpege/4dvarfr/ABCD/20190101T0000A/anyblock/demo.arpege-altitude.005.grib'

# Defining a constant (retrievable via genv)

The our example resource will be referenced in Genv by the 'DEMOFILE' key

In [11]:
from common.data.consts import GenvModelResource

class DemoConstante1(GenvModelResource):

    _footprint = dict(info = 'Constante 1',
                      attr = dict(kind = dict(values = ['ma_constante1']),
                                  gvar = dict(default = 'DEMOFILE')))
    
    @property
    def realkind(self):
        return 'democonstante'

Let's prepare the test:

In [12]:
# In real life, it will be done automatically by Olive or the job creation system
from gco.tools import genv
genv.register(cycle='test_cycle.01',
              DEMOFILE='the_test_constant.01')

The test itself:

In [13]:
rtest = toolbox.input(model='arpege', kind='ma_constante1',
                      # Provider + Container :
                      genv = 'test_cycle.01', incore=True)
rtest[0].location()

'gget://gco.meteo.fr/tampon/the_test_constant.01'

## And if it was a monthly constant ?

Or, the return of the predefined attribute...

In [14]:
from vortex.syntax.stdattrs import month_deco
from common.data.consts import GenvModelResource

class DemoConstante2(GenvModelResource):

    _footprint = [month_deco,
                  dict(info = 'Constante 2',
                       attr = dict(kind = dict(values = ['ma_constante2']),
                                   gvar = dict(default = 'DEMOFILE')))
                 ]
    
    @property
    def realkind(self):
        return 'democonstante'

Let's test the result:

In [15]:
rtest = toolbox.input(model='arpege', kind='ma_constante2', month=3,
                      # Provider + Container :
                      genv = 'test_cycle.01', incore=True)
rtest[0].location()

'gget://gco.meteo.fr/tampon/the_test_constant.01.m03'

If we look at the definition of the predefined attribute *month_deco*, we see that it is associated with a class decorator that alters the content of the **gget_basename** method: this is what explains the addition of '.m03' in the URI.